In [1]:
using JSON
using Random
using Distributions
using MySQL
using DotEnv
include("/home/peters/code/mapinator/estimation/bootstrap_functions.jl")
include("/home/peters/code/mapinator/estimation/functions.jl")
YEAR_INTERVAL = 2003:2023  # change this to select the years of data to include in the estimation
NUM = 5 # variable set it here
NUMSINKS = 6 #4 sinks are assumed in some of the code
sinks = ["Other Academic","Government","Private Sector","Teaching Universities","Postdoc", "Sessionals","Column Totals"]
sample = 10000;
iterate = 40
function name_to_oid(name, institution_mapping)
    for (key, value) in institution_mapping
        if value == name
            return key
        end
    end
    return "not found"
end
function run_sampler(placements,sample, cfg)
    d = DBInterface.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]));
    query = "insert into bootstrap_trials set trial_time=now(),sample_size=?,number_of_tiers=?"
    create_trial_statement =  DBInterface.prepare(d, query);
    query = "insert into bootstrap_data set trial_id=?,institution_id=?,estimated_tier=?"
    add_data_statement = DBInterface.prepare(d, query);
    academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer, institution_mapping = bootstrap_functions.sampler(placements, sample)
    institutions, out = functions.create_adjacency_matrix(academic_builder, sink_builder, academic,acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer);
    est_obj, est_alloc = functions.doit(out, collect(academic),collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), collect(postdoc), collect(lecturer),institutions, NUM, NUMSINKS);
    est_mat, est_count = functions.bucket_extract(est_alloc, out, NUM, NUMSINKS)
    t_table, t_allocation = functions.tiered_allocation(est_mat, est_count, institutions, est_alloc, NUM, NUMSINKS);
    c = DBInterface.execute(create_trial_statement, [sample, NUM])
    trial_id = DBInterface.lastrowid(c)
    for j in 1:NUM
        #println("Tier $j:")
        for (i, type) in enumerate(t_allocation)
            if type == j
                #println("  ", institutions[i], " (institution_id ", name_to_oid(institutions[i], institution_mapping), ")")
                cc = DBInterface.execute(add_data_statement, [trial_id, name_to_oid(institutions[i], institution_mapping), j])
            end
        end
    end
    DBInterface.close!(d)
    return institutions, t_allocation, institution_mapping, trial_id
end


run_sampler (generic function with 1 method)

In [2]:
#println(t_allocation)
#institutions, out = functions.create_adjacency_matrix(academic_builder, sink_builder, academic,acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer);

In [3]:
placements = bootstrap_functions.process_data(YEAR_INTERVAL);
cfg = DotEnv.config()

Threads.@threads for k in 1:iterate
    println("run = $k on thread $(Threads.threadid())")
    institutions, t_allocation, institution_mapping, trial_id = run_sampler(placements, sample, cfg)
    println("Finished trial_id ",trial_id)
end


run = 25 on thread 7
run = 33 on thread 9
run = 21 on thread 6
run = 1 on thread 1
run = 37 on thread 10
run = 9 on thread 3
run = 13 on thread 4
run = 5 on thread 2
run = 29 on thread 8
run = 17 on thread 5
FAULT: hiring 5 with graduating 2: downward rate: 10, upward rate: 16
FAULT: hiring 5 with graduating 1: downward rate: 1, upward rate: 3
FAULT: hiring 5 with graduating 3: downward rate: 35, upward rate: 37
FAULT: hiring 5 with graduating 4: downward rate: 2, upward rate: 6
FAULT: hiring 5 with graduating 3: downward rate: 17, upward rate: 29
Finished trial_id 263
run = 30 on thread 8
Finished trial_id 262
run = 38 on thread 10
FAULT: hiring 5 with graduating 3: downward rate: 13, upward rate: 15
Finished trial_id 264
run = 22 on thread 6
FAULT: hiring 5 with graduating 2: downward rate: 4, upward rate: 14
FAULT: hiring 5 with graduating 1: downward rate: 0, upward rate: 0
FAULT: hiring 5 with graduating 3: downward rate: 16, upward rate: 19
Finished trial_id 266
run = 2 on thread